In [ ]:
import json
import os

def preprocess_data(input_dir, output_file):
    data = []
    for file_name in os.listdir(input_dir):
        print(file_name)
        if file_name.endswith(".json"):
            with open(os.path.join(input_dir, file_name), 'r') as f:
                content = json.load(f)
                entry = {
                    "input": f"{content['genre']} | {content['type']} | {content['description']}",
                    "output": json.dumps(content["preset"]["settings"])
                }
                data.append(entry)
    with open(output_file, 'w') as f:
        json.dump(data, f, indent=4)

input_directory = "labeled_data_music"
output_data_file = "training_data.json"
preprocess_data(input_directory, output_data_file)

148.json
347.json
227.json
153.json
136.json
196.json
50.json
9.json
395.json
203.json
131.json
231.json
332.json
189.json
157.json
34.json
226.json
324.json
63.json
128.json
79.json
323.json
48.json
105.json
218.json
172.json
186.json
182.json
239.json
127.json
165.json
288.json
268.json
134.json
49.json
382.json
140.json
381.json
144.json
291.json
286.json
117.json
237.json
219.json
83.json
57.json
200.json
246.json
389.json
36.json
120.json
283.json
86.json
40.json
129.json
245.json
75.json
185.json
356.json
107.json
341.json
142.json
215.json
370.json
352.json
279.json
33.json
374.json
336.json
232.json
10.json
139.json
339.json
65.json
74.json
187.json
158.json
72.json
309.json
307.json
141.json
176.json
292.json
114.json
191.json
11.json
257.json
375.json
380.json
360.json
320.json
122.json
261.json
143.json
173.json
73.json
206.json
138.json
38.json
21.json
62.json
328.json
179.json
61.json
260.json
214.json
222.json
305.json
228.json
255.json
202.json
362.json
262.json
8.json
2

In [6]:
import json
import os

def preprocess_data(input_dir, output_file):
    data = []
    for file_name in os.listdir(input_dir):
        print(file_name)
        if file_name.endswith(".json"):
            with open(os.path.join(input_dir, file_name), 'r') as f:
                content = json.load(f)
                output_values = [value for key, value in content["preset"]["settings"].items() if isinstance(value, (int, float))]
                entry = {
                    "input": f"{content['genre']} | {content['type']} | {content['description']}",
                    "output": json.dumps(output_values)
                }
                data.append(entry)
    with open(output_file, 'w') as f:
        json.dump(data, f, indent=4)

input_directory = "labeled_data_music"
output_data_file = "training_data.json"
preprocess_data(input_directory, output_data_file)
# custom_warps, lfos, modulations, random_values, sample, wavetables

148.json
347.json
227.json
153.json
136.json
196.json
50.json
9.json
395.json
203.json
131.json
231.json
332.json
189.json
157.json
34.json
226.json
324.json
63.json
128.json
79.json
323.json
48.json
105.json
218.json
172.json
186.json
182.json
239.json
127.json
165.json
288.json
268.json
134.json
49.json
382.json
140.json
381.json
144.json
291.json
286.json
117.json
237.json
219.json
83.json
57.json
200.json
246.json
389.json
36.json
120.json
283.json
86.json
40.json
129.json
245.json
75.json
185.json
356.json
107.json
341.json
142.json
215.json
370.json
352.json
279.json
33.json
374.json
336.json
232.json
10.json
139.json
339.json
65.json
74.json
187.json
158.json
72.json
309.json
307.json
141.json
176.json
292.json
114.json
191.json
11.json
257.json
375.json
380.json
360.json
320.json
122.json
261.json
143.json
173.json
73.json
206.json
138.json
38.json
21.json
62.json
328.json
179.json
61.json
260.json
214.json
222.json
305.json
228.json
255.json
202.json
362.json
262.json
8.json
2

In [8]:
pip install transformers sentencepiece torch accelerate evaluate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 32.8 kB/s eta 0:00:0031m41.3 kB/s eta 0:00:05
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24951 sha256=4f06ca812ebbc6a527880ee2e4986316ec5aeabe9fc83924d84262ec2dfb0b6c
  Stored in directory: /home/drama/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import json
from torch.utils.data import Dataset

class SynthPresetDataset(Dataset):
    def __init__(self, data_file, tokenizer, max_length=512):
        with open(data_file, 'r') as f:
            self.data = json.load(f)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        input_enc = self.tokenizer(item["input"], max_length=self.max_length, truncation=True, padding="max_length", return_tensors="pt")
        output_enc = self.tokenizer(item["output"], max_length=self.max_length, truncation=True, padding="max_length", return_tensors="pt")
        return {
            "input_ids": input_enc.input_ids.squeeze(),
            "attention_mask": input_enc.attention_mask.squeeze(),
            "labels": output_enc.input_ids.squeeze()
        }

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

dataset = SynthPresetDataset("training_data.json", tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps=10,
    save_total_limit=2,
    evaluation_strategy="no",
    logging_dir="./logs"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

# 312bcaf95323b58c7785fb87c1bf62211880ca96
trainer.train()
tokenizer.save_pretrained("./results/final")


2024-12-02 13:39:04.666022: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-02 13:39:04.675505: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733135944.687001 1033991 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733135944.690602 1033991 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 13:39:04.707125: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

  0%|          | 0/150 [00:00<?, ?it/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


{'train_runtime': 2028.5367, 'train_samples_per_second': 0.59, 'train_steps_per_second': 0.074, 'train_loss': 2.5594342041015623, 'epoch': 3.0}


TrainOutput(global_step=150, training_loss=2.5594342041015623, metrics={'train_runtime': 2028.5367, 'train_samples_per_second': 0.59, 'train_steps_per_second': 0.074, 'total_flos': 162004136361984.0, 'train_loss': 2.5594342041015623, 'epoch': 3.0})

wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.


In [2]:
import json
import os
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_path = "./results/checkpoint-150"
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained(model_path)


def generate_preset(description, model, tokenizer, output_file, max_length=512):
    input_text = f"{description}"
    input_ids = tokenizer(input_text, return_tensors="pt", max_length=max_length, truncation=True).input_ids
    outputs = model.generate(input_ids, max_length=max_length)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Decoded Output:", decoded_output)
    preset_settings = json.loads(decoded_output)
    
    vital_data = {
        "preset_name": "TEST1",
        "preset_style": "",
        "settings": preset_settings,
        "synth_version": "1.5.5"
    }
    
    with open(output_file, 'w') as f:
        json.dump(vital_data, f, indent=4)
    print(f"Preset saved to {output_file}")

description = "buzzy, distorted bass with a heavy kick, ideal for intense moments"
output_file = "preset.vital"
generate_preset(description, model, tokenizer, output_file)


2024-12-02 18:43:50.539671: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-02 18:43:50.748309: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733154230.827506 1107063 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733154230.849591 1107063 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 18:43:51.052573: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Decoded Output: "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "Like, "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread": "spread":


JSONDecodeError: Extra data: line 1 column 9 (char 8)

In [ ]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 1. Load and preprocess the data
with open("training_data.json", "r") as f:
    data = json.load(f)

input_texts = [item["input"] for item in data]
output_texts = [item["output"] for item in data]

# Add start and end tokens to the output text
output_texts = ["<start> " + text + " <end>" for text in output_texts]

# Tokenization
input_tokenizer = Tokenizer()
input_tokenizer.fit_on_texts(input_texts)
input_sequences = input_tokenizer.texts_to_sequences(input_texts)
input_vocab_size = len(input_tokenizer.word_index) + 1

output_tokenizer = Tokenizer()
output_tokenizer.fit_on_texts(output_texts)
output_sequences = output_tokenizer.texts_to_sequences(output_texts)
output_vocab_size = len(output_tokenizer.word_index) + 1

# Padding
max_encoder_seq_length = max([len(seq) for seq in input_sequences])
max_decoder_seq_length = max([len(seq) for seq in output_sequences])

encoder_input_data = pad_sequences(input_sequences, maxlen=max_encoder_seq_length, padding="post")
decoder_input_data = pad_sequences(output_sequences, maxlen=max_decoder_seq_length, padding="post")

# Prepare decoder target data as categorical labels (not one-hot encoded)
decoder_target_data = np.zeros((len(output_sequences), max_decoder_seq_length), dtype="int32")
for i, seq in enumerate(output_sequences):
    for t, word_id in enumerate(seq[1:]):  # Skip the start token
        decoder_target_data[i, t] = word_id

# 2. Split data into training and validation sets
encoder_input_train, encoder_input_val, decoder_input_train, decoder_input_val, decoder_target_train, decoder_target_val = train_test_split(
    encoder_input_data, decoder_input_data, decoder_target_data, test_size=0.2
)

# 3. Build the Seq2Seq model
# Encoder
encoder_inputs = Input(shape=(max_encoder_seq_length,))
encoder_embedding = Embedding(input_vocab_size, 256)(encoder_inputs)
encoder_lstm = LSTM(256, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_decoder_seq_length,))
decoder_embedding = Embedding(output_vocab_size, 256)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(output_vocab_size, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

# 4. Train the model
model.fit(
    [encoder_input_train, decoder_input_train],
    decoder_target_train,
    batch_size=64,
    epochs=50,
    validation_data=([encoder_input_val, decoder_input_val], decoder_target_val),
)

# 5. Save the model
model.save("seq2seq_vital.h5")

# 6. Define inference models for testing
# Encoder model for inference
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder model for inference
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_lstm_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

2024-12-02 14:58:52.882715: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [ ]:
# 7. Function for generating JSON outputs from input text
def generate_json(input_text):
    input_seq = input_tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=max_encoder_seq_length, padding="post")

    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = output_tokenizer.word_index["<start>"]

    stop_condition = False
    generated_sequence = ""

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = output_tokenizer.index_word.get(sampled_token_index, "")

        if sampled_token == "<end>" or len(generated_sequence) > max_decoder_seq_length:
            stop_condition = True
        else:
            generated_sequence += sampled_token

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        states_value = [h, c]

    return generated_sequence

# Example usage:
print(generate_json("Multigenre | Seq | buzzy, distorted bass..."))

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset
from evaluate import load
import json

# Загрузка и подготовка данных
data_file = "training_data.json"
dataset = load_dataset("json", data_files=data_file)

# Загрузка токенизатора и модели
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Токенизация данных
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["input"],
        max_length=512,
        truncation=True,
        padding="max_length"
    )
    labels = tokenizer(
        examples["output"],
        max_length=512,
        truncation=True,
        padding="max_length"
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Преобразование набора данных
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Разделение на обучающую и тестовую выборки
data_split = tokenized_datasets["train"].train_test_split(test_size=0.2)
train_dataset = data_split["train"]
val_dataset = data_split["test"]

# Определение вычисления метрик
rouge = load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return rouge.compute(predictions=decoded_preds, references=decoded_labels)

# Data Collator для Seq2Seq задач
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Параметры тренировки
training_args = TrainingArguments(
    output_dir="./results2",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    evaluation_strategy="steps",
    eval_steps=100,
    save_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    logging_dir="./logs2",
    logging_steps=50,
    learning_rate=5e-5,
    warmup_steps=500,
    weight_decay=0.01
)

# Инициализация Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Обучение
trainer.train()

# Сохранение модели и токенизатора
trainer.save_model("./results2/final_model")
tokenizer.save_pretrained("./results2/final_model")

print("Training complete and model saved.")


2024-12-02 22:59:35.759471: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-02 22:59:35.968716: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733169576.047933 1252176 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733169576.070278 1252176 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 22:59:36.270896: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

  0%|          | 0/120 [00:00<?, ?it/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


{'loss': 7.0054, 'grad_norm': 34.599849700927734, 'learning_rate': 5e-06, 'epoch': 1.25}
{'loss': 4.2946, 'grad_norm': 1.7117702960968018, 'learning_rate': 1e-05, 'epoch': 2.5}


  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
import json
import os
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_path = "./results/checkpoint-150"
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained(model_path)


def generate_preset(description, model, tokenizer, output_file, max_length=512):
    input_text = f"{description}"
    input_ids = tokenizer(input_text, return_tensors="pt", max_length=max_length, truncation=True).input_ids
    outputs = model.generate(input_ids, max_length=max_length)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Decoded Output:", decoded_output)
    preset_settings = json.loads(decoded_output)
    
    vital_data = {
        "preset_name": "TEST1",
        "preset_style": "",
        "settings": preset_settings,
        "synth_version": "1.5.5"
    }
    
    with open(output_file, 'w') as f:
        json.dump(vital_data, f, indent=4)
    print(f"Preset saved to {output_file}")

description = "buzzy, distorted bass with a heavy kick, ideal for intense moments"
output_file = "preset.vital"
generate_preset(description, model, tokenizer, output_file)
